In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
file_path = '../data/combined_keypoints.csv'
df = pd.read_csv(file_path)

le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])

X = df.drop('Class', axis=1).values
y = df['Class'].values

def convert_to_geometric(data, labels):
    data_list = []
    for i in range(len(data)):
        x = torch.tensor(data[i].reshape(-1, 3), dtype=torch.float)
        edge_index = torch.tensor([[i, i+1] for i in range(len(x)-1)], dtype=torch.long).t().contiguous()
        y = torch.tensor(labels[i], dtype=torch.long)
        data_list.append(Data(x=x, edge_index=edge_index, y=y))
    return data_list

data_list = convert_to_geometric(X, y)

In [ ]:
train_data, test_data = train_test_split(data_list, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

/home/abdulah/anaconda3/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
class YogaPoseGNN(torch.nn.Module):
    def __init__(self, num_classes, hidden_dim=64, num_layers=3, dropout_rate=0.5):
        super(YogaPoseGNN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(3, hidden_dim))
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        self.fc1 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, num_classes)
        self.dropout = torch.nn.Dropout(p=dropout_rate)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.dropout(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(model, optimizer, criterion, train_loader):
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(model, loader):
    model.eval()
    correct = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data in loader:
            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == data.y).sum().item()
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.cpu().numpy())
    return correct / len(loader.dataset), all_preds, all_labels

In [ ]:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.2, 0.7)
    num_layers = trial.suggest_int('num_layers', 2, 5)
    hidden_dim = trial.suggest_int('hidden_dim', 32, 128)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])
    step_size = trial.suggest_int('step_size', 10, 50)
    gamma = trial.suggest_uniform('gamma', 0.1, 0.9)

    model = YogaPoseGNN(num_classes=len(le.classes_), hidden_dim=hidden_dim, num_layers=num_layers, dropout_rate=dropout_rate)

    if optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.9)

    criterion = torch.nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

      train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    for epoch in range(1, 101):
        train_loss = train(model, optimizer, criterion, train_loader)
        train_acc, _, _ = test(model, train_loader)
        test_acc, _, _ = test(model, test_loader)
        scheduler.step()

    return 1 - test_acc

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print(f'Best parameters: {study.best_params}')
print(f'Best accuracy: {1 - study.best_value}')

[I 2024-08-04 16:59:08,881] A new study created in memory with name: no-name-8780294e-88ba-4517-9f0a-ea218244c8e4
/tmp/ipykernel_20421/515450261.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
/tmp/ipykernel_20421/515450261.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)
/tmp/ipykernel_20421/515450261.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.2

KeyboardInterrupt: 

In [ ]:
Trial 29 finished with value: 0.6325564457392572 and parameters: {'lr': 0.006899563880774035, 'weight_decay': 2.610264590093146e-05, 'dropout_rate': 0.20591923852703245, 'num_layers': 4, 'hidden_dim': 128, 'batch_size': 64, 'optimizer': 'SGD', 'step_size': 46, 'gamma': 0.2661927678915462}. Best is trial 29 with value: 0.6325564457392572.

In [ ]:
print(f'Best parameters: {study.best_params}')

Best parameters: {'lr': 0.0005253349961281477, 'weight_decay': 1.776618474427631e-05, 'dropout_rate': 0.3636132094412057, 'num_layers': 5, 'hidden_dim': 127, 'batch_size': 16, 'optimizer': 'Adam', 'step_size': 50, 'gamma': 0.2692244103661817}


In [ ]:
best_params = study.best_params
model = YogaPoseGNN(num_classes=len(le.classes_), hidden_dim=best_params['hidden_dim'], num_layers=best_params['num_layers'], dropout_rate=best_params['dropout_rate'])

if best_params['optimizer'] == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'], momentum=0.9)

criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=best_params['step_size'], gamma=best_params['gamma'])

train_loader = DataLoader(train_data, batch_size=best_params['batch_size'], shuffle=True)
test_loader = DataLoader(test_data, batch_size=best_params['batch_size'], shuffle=False)

for epoch in range(1, 201):
    train_loss = train(model, optimizer, criterion, train_loader)
    train_acc, train_preds, train_labels = test(model, train_loader)
    test_acc, test_preds, test_labels = test(model, test_loader)
    scheduler.step()

    print(f'Epoch {epoch+1}: Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch 2: Loss: 4.2192, Train Acc: 0.0536, Test Acc: 0.0528
Epoch 3: Loss: 3.9639, Train Acc: 0.0870, Test Acc: 0.0914
Epoch 4: Loss: 3.6746, Train Acc: 0.1182, Test Acc: 0.1245
Epoch 5: Loss: 3.5122, Train Acc: 0.1279, Test Acc: 0.1351
Epoch 6: Loss: 3.4285, Train Acc: 0.1487, Test Acc: 0.1395
Epoch 7: Loss: 3.3680, Train Acc: 0.1417, Test Acc: 0.1482
Epoch 8: Loss: 3.3235, Train Acc: 0.1572, Test Acc: 0.1577
Epoch 9: Loss: 3.2796, Train Acc: 0.1595, Test Acc: 0.1642
Epoch 10: Loss: 3.2394, Train Acc: 0.1769, Test Acc: 0.1737
Epoch 11: Loss: 3.1982, Train Acc: 0.1964, Test Acc: 0.1948
Epoch 12: Loss: 3.1606, Train Acc: 0.2067, Test Acc: 0.1923
Epoch 13: Loss: 3.1285, Train Acc: 0.2084, Test Acc: 0.2014
Epoch 14: Loss: 3.0969, Train Acc: 0.2174, Test Acc: 0.2156
Epoch 15: Loss: 3.0687, Train Acc: 0.2278, Test Acc: 0.2203
Epoch 16: Loss: 3.0443, Train Acc: 0.2281, Test Acc: 0.2072
Epoch 17: Loss: 3.0018, Train Acc: 0.2183, Test Acc: 0.2145
Epoch 18: Loss: 2.9864, Train Acc: 0.2217, Test 

In [ ]:
print("Classification Report:")
print(classification_report(test_labels, test_preds))

Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.45      0.40        22
           1       0.45      0.41      0.43        49
           2       0.53      0.22      0.31        37
           3       0.52      0.42      0.46        38
           4       0.58      0.59      0.58        32
           5       0.50      0.46      0.48        28
           6       0.62      0.59      0.61        39
           7       0.23      0.32      0.27        28
           8       0.36      0.51      0.43        39
           9       0.58      0.71      0.64        97
          10       0.11      0.04      0.06        23
          11       0.52      0.33      0.41        36
          12       0.53      0.50      0.52        32
          13       0.56      0.50      0.53        48
          14       0.50      0.55      0.52        20
          15       0.56      0.39      0.46        23
          16       0.66      0.58      0.61        33
    

In [ ]:
conf_matrix = confusion_matrix(test_labels, test_preds)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[10  0  0 ...  0  0  0]
 [ 0 20  0 ...  0  1  0]
 [ 0  0  8 ...  0  0  0]
 ...
 [ 0  0  0 ...  8  0  1]
 [ 0  0  0 ...  0  6  0]
 [ 0  0  0 ...  0  1  8]]
